In [18]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sqlalchemy import create_engine

In [12]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df = pd.read_sql_query('select * from useducation',con=engine)
engine.dispose()

In [13]:
df.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [14]:
# Filling in null values
filter_list = ['ENROLL', 'TOTAL_REVENUE', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE', 'TOTAL_EXPENDITURE',
               'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE',
               'GRADES_PK_G', 'GRADES_KG_G', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G', 'GRADES_9_12_G', 'GRADES_ALL_G']

states = df['STATE'].unique()

for state in states:
    df.loc[df['STATE'] == state, filter_list] = df.loc[df['STATE'] == state, filter_list].interpolate()

# Dropping null values
df.dropna(inplace=True)

Question 1<br>
Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!.

In [15]:
df['WEIGHTED_AVG'] = (df['GRADES_4_G'] * ((df['AVG_MATH_4_SCORE'] + df['AVG_READING_4_SCORE']) / 2) + 
                       df['GRADES_8_G'] * ((df['AVG_MATH_8_SCORE'] + df['AVG_READING_8_SCORE']) / 2)
                      ) / (df['GRADES_4_G'] + df['GRADES_8_G'])

Question 2<br>
What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [16]:
df[['WEIGHTED_AVG', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']].corr()

,WEIGHTED_AVG,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
WEIGHTED_AVG,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


The closest correlated type would be Instruction Expenditure.

Question 3<br>
Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [26]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X = df[['INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE',
        'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']]

X = StandardScaler().fit_transform(X)

sklearn_pca = PCA(n_components=1)
df['PCA_1ST'] = sklearn_pca.fit_transform(X)

sklearn_pca.explained_variance_ratio_

array([0.9430175])

94% of the total variance is explained by teh 1st component.

Question 4<br>
What is the correlation between the overall score variable and the 1st principal component? 

In [27]:
df[['INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE',
    'PCA_1ST', 'WEIGHTED_AVG']].corr()

,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,PCA_1ST,WEIGHTED_AVG
INSTRUCTION_EXPENDITURE,1.000000,0.974399,0.902173,0.884798,0.968606,0.087386
SUPPORT_SERVICES_EXPENDITURE,0.974399,1.000000,0.952282,0.911245,0.988345,0.079002
OTHER_EXPENDITURE,0.902173,0.952282,1.000000,0.918188,0.971343,0.010959
CAPITAL_OUTLAY_EXPENDITURE,0.884798,0.911245,0.918188,1.000000,0.955792,-0.005389
PCA_1ST,0.968606,0.988345,0.971343,0.955792,1.000000,0.044596
WEIGHTED_AVG,0.087386,0.079002,0.010959,-0.005389,0.044596,1.000000


The correlation between the overall score variable and the 1st principal component is .0044596, much less then the other Expenditures.

Question 5<br>
If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

I would use 1st principal component instead of the other variables, it contains over 95% of the data from the other expenditures.